In [1]:
PATH = '/content/drive/My Drive/NLP/'

In [3]:
import os
import json
import numpy as np

In [4]:
def decode_cat(res):
    ans = []
    for r in res:
        temp = [categories[i] for i in r]
        ans.append(temp)
    return ans

In [5]:
with open(PATH+'categories.json') as f:
  categories = json.load(f)

tweets = []
for line in open(PATH+'train_gold.json', 'r'):
    tweets.append(json.loads(line))

test = []
for line in open(PATH+'test_unlabeled.json', 'r'):
    test.append(json.loads(line))
    
dev = []
for line in open(PATH+'dev_unlabeled.json','r'):
  dev.append(json.loads(line))

Note: use `np.average(array,axis,weights)` to fine-tune weights

In [6]:
_pred = []
_pred_t = []
model = ['pred_distilroberta_ensemble0-15','pred_distilbert_ensemble0-4','pred_bert_uncased_ensemble0-10','pred_roberta_ensemble0-31','pred_xlnet_ensemble0-6']

for m in model:
  with open(PATH+m+'_d.npy', 'rb') as f:
    a = np.load(f)
    _pred.append(a)
  with open(PATH+m+'_t.npy', 'rb') as f:
    a = np.load(f)
    _pred_t.append(a)

In [92]:
with open(PATH+'LSTM_text_d.npy', 'rb') as f:
  lstm = np.load(f)
with open(PATH+'LSTM_text_t.npy', 'rb') as f:
  lstm_t = np.load(f)

Grid search

In [7]:
for i in range(100):
  weights = np.random.dirichlet(np.ones(5),size=1)[0]
  pred = np.average(_pred,axis=0,weights=weights)
  #pred_t = np.average(_pred_t,axis=0,weights=weights)
  '''for idx, d in enumerate(dev):
    if d['reply']=='':
      pred[idx] = 0.7*pred[idx]+0.3*lstm[idx]'''
  results = [np.argsort(res)[::-1][:6] for res in pred]
  results = decode_cat(results)
  for idx,t in enumerate(dev):
    t['categories'] = results[idx]
  weights = (np.round(weights,2)*100).astype(int)
  if not os.path.isfile(PATH+'dev{}-{}-{}-{}-{}.json'.format(weights[0],weights[1],weights[2],weights[3],weights[4])):
    for i in dev:
      with open(PATH+'dev{}-{}-{}-{}-{}.json'.format(weights[0],weights[1],weights[2],weights[3],weights[4]), 'a', encoding='utf-8') as f:
        json.dump(i, f, ensure_ascii=False)
        f.write('\n')

# Ensemble Final

In [93]:
weights = [0.07,0.07,0.34,0.22,0.28]
pred = np.average(_pred,axis=0,weights=weights)
pred_t = np.average(_pred_t,axis=0,weights=weights)
for idx, d in enumerate(dev):
  if d['reply']=='':
    pred[idx] = 0.7*pred[idx]+0.3*lstm[idx]
for idx, d in enumerate(test):
  if d['reply']=='':
    pred_t[idx] = 0.7*pred_t[idx]+0.3*lstm_t[idx]

In [94]:
results = [np.argsort(res)[::-1][:6] for res in pred]
results = decode_cat(results)

results_t = [np.argsort(res)[::-1][:6] for res in pred_t]
results_t = decode_cat(results_t)

In [95]:
for idx,t in enumerate(dev):
    t['categories'] = results[idx]
for idx,t in enumerate(test):
    t['categories'] = results_t[idx]

In [96]:
for i in dev:
  with open('dev.json', 'a', encoding='utf-8') as f:
    json.dump(i, f, ensure_ascii=False)
    f.write('\n')
for i in test:
  with open('eval.json', 'a', encoding='utf-8') as f:
    json.dump(i, f, ensure_ascii=False)
    f.write('\n')